In [27]:
import sys
import xlsxwriter
import pandas as pd
import sys
import psycopg2
import os
from dotenv import load_dotenv
from datetime import date


In [35]:
today = date.today()
load_dotenv()
#configurar la conexion a la base de datos
DB_USERNAME = 'djacome'
DB_DATABASE = 'encuesta'
DB_PASSWORD = 'V+hD2%jd27YN'
DB_PORT = '5432'
DB_HOST='192.168.0.221'

# Conectar a DB
# Conectar a PostgreSQL
try:
    cnx = psycopg2.connect(
        user=DB_USERNAME,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        database=DB_DATABASE
    )
    print("Conexión exitosa")
except psycopg2.Error as e:
    print("Ocurrio un error al conectar a la base de datos", e)



Conexión exitosa


In [38]:
encuestas=pd.read_sql("""select respuestas20.aplica, egresados.cuenta, respuestas20.fec_capt, carreras.carrera, carreras.plantel
                         from((respuestas20
                         inner join egresados on respuestas20.cuenta=egresados.cuenta)
                         inner join carreras  on carreras.clave_carrera=egresados.carrera and carreras.clave_plantel=egresados.plantel)
                         
                         where (respuestas20.completed=1) and egresados.anio_egreso=2020""",cnx)


C:\Users\dperez\AppData\Local\Temp\ipykernel_31964\1785139216.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  encuestas=pd.read_sql("""select respuestas20.aplica, egresados.cuenta, respuestas20.fec_capt, carreras.carrera, carreras.plantel


In [39]:
ClavesNombres = {'17':'Erendira', '12':'Monica', '15':'César', '20':'María', '21':'Ivonne', '8':'Elia', '7':'otra monica','6':'Silvia','9':'Veronica',
                 '14':'Alberto','18':'Daniela','19':'Elvira','13':'Carolina','22':'Elizabeth','23':'Sandra','24':'Miguel','25':'Amanda'}


In [40]:
def mapeo(x):
    if(x==None):
        return 'INTERNET'
    else:
        try:
            return ClavesNombres[x]
        except:
            return 'Encuestador Desconocido'

In [41]:
encuestas['aplica'] = encuestas['aplica'].map(lambda x:mapeo(x))

print(encuestas['fec_capt'])
print(encuestas[0:10])
print(encuestas['aplica'].unique())

writer = pd.ExcelWriter('storage/reporte_individual20.xlsx', engine='xlsxwriter')

workbook = writer.book
a_color='#173d83'

0      2024-04-24 00:00:00+00:00
1      2025-03-04 00:00:00+00:00
2      2024-05-08 00:00:00+00:00
3      2024-09-23 23:40:49+00:00
4      2024-11-21 00:00:00+00:00
                  ...           
4111   2024-10-17 00:00:00+00:00
4112                         NaT
4113   2024-10-15 00:00:00+00:00
4114                         NaT
4115   2024-04-25 00:00:00+00:00
Name: fec_capt, Length: 4116, dtype: datetime64[ns, UTC]
                    aplica     cuenta                  fec_capt  \
0                    César  314025834 2024-04-24 00:00:00+00:00   
1                 Erendira  309266880 2025-03-04 00:00:00+00:00   
2                   Sandra  412528703 2024-05-08 00:00:00+00:00   
3  Encuestador Desconocido  312316084 2024-09-23 23:40:49+00:00   
4                   Amanda  313129252 2024-11-21 00:00:00+00:00   
5                   Miguel  416013788 2024-09-18 00:00:00+00:00   
6  Encuestador Desconocido  313234295 2024-09-18 23:30:43+00:00   
7  Encuestador Desconocido  313292134 2024-0

In [43]:
#estilos------------------------------

negro_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':13}) 

header_format = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'font_size':12})
blue_content = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':10,
    'border_color':a_color})
date_content = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':10,
    'border_color':a_color,
    'num_format': 'dd/mm/yy'})
date_content_bold = workbook.add_format({
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':12,
    'bold': True,
    'num_format': 'dd/mm/yy'})

In [44]:
worksheet =workbook.add_worksheet()
worksheet.merge_range('C2:H3', 'PROGRAMA DE VINCULACION A EGRESADOS UNAM', negro_b)
worksheet.merge_range('C4:H4', 'CONSECUTIVO ENCUESTAS 2020', negro_b)
worksheet.insert_image("A1", "img/logoPVE.png",{"x_scale": 0.2, "y_scale": 0.2})
worksheet.merge_range('G6:H6',today, date_content_bold)
worksheet.write('B8','Numero de cuenta',header_format)
worksheet.write('C8','Fecha en que realizó',header_format)
worksheet.write('D8','Aplicador',header_format)
worksheet.write('E8','Carrera',header_format)
worksheet.write('F8','Plantel',header_format)

C:\Users\dperez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xlsxwriter\worksheet.py:1574: UserWarning: Image file 'img/logoPVE.png' not found.
  warn(f"Image file '{filename}' not found.")


0

In [45]:
for i in range(0,len(encuestas)):
    worksheet.write('B'+str(i+9),encuestas['cuenta'].values[i],blue_content)
    worksheet.write('C'+str(i+9),str(encuestas['fec_capt'].values[i]),date_content)
    worksheet.write('D'+str(i+9),encuestas['aplica'].values[i],blue_content)
    worksheet.write('E'+str(i+9),encuestas['carrera'].values[i],blue_content)
    worksheet.write('F'+str(i+9),encuestas['plantel'].values[i],blue_content)
worksheet.set_column('J:J',15)
worksheet.set_column('B:C',17)
worksheet.set_column('E:F',28)
worksheet.set_landscape()
worksheet.set_paper(9)
worksheet.fit_to_pages(1, 1)  
workbook.close()
